# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

vacation_df = pd.read_csv("Output_Data/cities.csv")

vacation_df

,City ID,City,Cloudiness,Country,Humidity,Lat,Lng,Temperature,Wind Speed
0,5536630,Cedar City,100,US,48,37.6775,-113.0619,26.89,2.22
1,2020744,Listvyanka,25,RU,91,51.8535,104.8693,12.54,1.93
2,338345,Edd,100,ER,69,13.9297,41.6925,32.57,0.90
3,6165406,Thompson,20,CA,46,55.7435,-97.8558,22.09,3.09
4,4031574,Provideniya,16,RU,60,64.3833,-173.3000,7.17,0.70
...,...,...,...,...,...,...,...,...,...
580,2036713,Huanren,100,CN,100,41.2647,125.3667,21.42,2.28
581,3141310,Rørvik,20,NO,77,64.8621,11.2373,17.06,3.09
582,2122605,Okhotsk,100,RU,95,59.3833,143.3000,13.34,3.68
583,3653523,Palora,78,EC,52,-1.7000,-77.9333,28.06,2.28


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
fig=gmaps.figure()

heat=gmaps.heatmap_layer(vacation_df[["Lat","Lng"]],vacation_df["Humidity"])

fig.add_layer(heat)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_con_df = vacation_df.loc[(vacation_df["Temperature"] >=20) & (vacation_df["Temperature"] <=35) & (vacation_df["Wind Speed"] <=6) & (vacation_df["Cloudiness"] <=3) & (vacation_df["Humidity"] <=30)]
hotel_df = weather_con_df
hotel_df

,City ID,City,Cloudiness,Country,Humidity,Lat,Lng,Temperature,Wind Speed
30,2509835,Valverde del Camino,0,ES,26,37.5751,-6.7543,23.91,4.31
50,3464008,Esmeraldas,0,BR,23,-19.7625,-44.3139,25.36,3.24
64,1529484,Kumul,0,CN,22,42.8000,93.4500,26.92,4.70
127,3459785,João Pinheiro,0,BR,22,-17.7425,-46.1725,28.02,2.36
145,3446077,Três Pontas,0,BR,27,-21.3667,-45.5125,25.40,1.55
152,3382160,Cayenne,0,GF,1,4.9333,-52.3333,28.94,5.66
202,2212775,Sabha,0,LY,15,27.0377,14.4283,31.69,3.74
204,2520610,Caceres,0,ES,20,39.5220,-6.3748,28.11,4.16
291,5887798,Assiniboia,0,CA,29,49.6334,-105.9845,30.90,3.09
326,304919,Malatya Province,0,TR,28,38.5000,38.0000,27.03,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
for index, row in hotel_df.iterrows():
    
    print(index)
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    lat = row["Lat"]
    lng = row["Lng"]    
        
    params={"location": f"{lat},{lng}",
                "types": "lodging",
                "radius": 5000,
                "key": g_key,
                }

        
    params["location"] = f"{lat}, {lng}"
        
    response = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]

    except (KeyError, IndexError):
        
        hotel_df.loc[index,"Hotel Name"] = "No Data"
    
hotel_df

30


C:\Users\NITIN\AppData\Local\Temp\ipykernel_4688\176109135.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index,"Hotel Name"] = "No Data"


50
64
127
145
152
202
204
291
326
337
368
487
515
540
548
559


,City ID,City,Cloudiness,Country,Humidity,Lat,Lng,Temperature,Wind Speed,Hotel Name
30,2509835,Valverde del Camino,0,ES,26,37.5751,-6.7543,23.91,4.31,No Data
50,3464008,Esmeraldas,0,BR,23,-19.7625,-44.3139,25.36,3.24,No Data
64,1529484,Kumul,0,CN,22,42.8000,93.4500,26.92,4.70,No Data
127,3459785,João Pinheiro,0,BR,22,-17.7425,-46.1725,28.02,2.36,No Data
145,3446077,Três Pontas,0,BR,27,-21.3667,-45.5125,25.40,1.55,No Data
152,3382160,Cayenne,0,GF,1,4.9333,-52.3333,28.94,5.66,No Data
202,2212775,Sabha,0,LY,15,27.0377,14.4283,31.69,3.74,No Data
204,2520610,Caceres,0,ES,20,39.5220,-6.3748,28.11,4.16,No Data
291,5887798,Assiniboia,0,CA,29,49.6334,-105.9845,30.90,3.09,No Data
326,304919,Malatya Province,0,TR,28,38.5000,38.0000,27.03,3.60,No Data


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Valverde del Camino</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Esmeraldas</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Kumul</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>João Pinheiro</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Três Pontas</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Cayenne</dd>\n<dt>Country</dt><dd>GF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Sabha</dd>\n<dt>Country</dt><dd>LY</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Caceres</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Data</dd>\n<dt>City</dt><dd>Assiniboia</dd>\n<dt>Country</dt>

In [9]:
type(locations)


pandas.core.frame.DataFrame

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))